### 1. Settings

In [1]:
#####################################
##########  DEPENDECIES ############
#####################################

import os
import pickle
from tqdm import tqdm # type: ignore
from datetime import date

import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import utils.prep as pr
import utils.eval as ev
import utils.inference as infer
from utils.sampling import create_splits, prep_cv_validation
from utils.training import cv_cluster_set, cv_training_epochs_sets
from utils.training import results_dict_todf, cv_step_2, full_step_2

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
#####################################
############  CONSTANTS #############
#####################################
RS = 42

MODEL = "CodeT5"
BATCH_SIZE = 16
DECODER_LENGTH = 20
ENCODER_LENGTH = 30
ANALYSIS_POSTFIX = f"mined_no_drift_{str(date.today())}"
DATE_STR = 20240721
SEMANTIC_DRIFT = True
model_name="Salesforce/codet5-base-multi-sum"

FULL_TRAIN_ARGS = {
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "MODEL": MODEL,
    "SEQ_TRAINER_ARGS": {
        "overwrite_output_dir": True,
        "num_train_epochs": [0, 1, 5, 8, 10, 16],
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 6e-6,
        "warmup_steps": 500,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "no",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
        "output_dir" : 'reports/results',
        "logging_dir" : "reports/logs",
    },
}

tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rouge = evaluate.load('rouge')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Conala data. Preprocessing. 

In [2]:
experiment_config = {
    "DATE_STR" : "20240721",
    "RS" : 42,
    "DRIFT_TYPE" : "sudden",
    "NFOLD" : 3,
    "FULL_TRAIN_ARGS" : FULL_TRAIN_ARGS,
    "MODEL_NAME" : model_name,
    "ANALYSIS_POSTFIX" : ANALYSIS_POSTFIX,
    "CLUSTER_EPOCHS" : 1,
}

In [3]:
sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, test=False)
train_dataset, test_data, test_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"]

splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
                            experiment_config=experiment_config)

Train Data:  (7942, 11)
Test Data:  (2058, 11)
Train Data: Cluster cluster
2    3632
3    2204
1    1672
0     414
4      20
Name: count, dtype: int64
Test Data: Cluster cluster
4    1980
3      39
2      25
1      12
0       2
Name: count, dtype: int64


Filter:   0%|          | 0/2058 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2058 [00:00<?, ? examples/s]

Map:   0%|          | 0/2058 [00:00<?, ? examples/s]

Fold 0
Fold 1
Fold 2


In [5]:
fold_results = cv_training_epochs_sets(experiment_config=experiment_config,
                            splits=splits,
                            questions_list=questions_list,
                            train_dataset=train_dataset,
                            tokenizer=tokenizer)

Fold 0


Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5298 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5298 [00:00<?, ? examples/s]

...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.937100,4.532994,0.264300,0.070400,0.232800,0.233000,13.984200,0.023800,1.000000,1.130300,28574,25280
2,3.949500,4.521092,0.266700,0.070500,0.234300,0.234600,13.960800,0.023500,1.000000,1.131200,28596,25280


TRAINING EPOCH SET 16
TRAINING EPOCHS 6
LOADING MODEL ./tmp/
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.676500,4.609656,0.259800,0.068200,0.228300,0.228600,13.983400,0.023000,1.000000,1.125200,28446,25280
2,3.790000,4.533356,0.267100,0.071800,0.234400,0.234500,13.784200,0.024400,1.000000,1.113100,28140,25280
3,3.935400,4.508908,0.269600,0.072400,0.236500,0.236600,14.063400,0.025000,1.000000,1.140800,28840,25280
4,3.980200,4.499930,0.266900,0.071400,0.235000,0.235100,13.969400,0.024300,1.000000,1.132200,28621,25280
5,3.964000,4.499954,0.267300,0.071500,0.234500,0.234500,14.120300,0.024500,1.000000,1.145300,28953,25280
6,3.939900,4.503712,0.266900,0.071500,0.234200,0.234200,14.041500,0.024800,1.000000,1.138800,28789,25280


In [11]:
for cluster_idx in [1, 4, 3]:
    fold_results = cv_cluster_set(experiment_config=experiment_config,
                                    splits=splits,
                                    questions_list=questions_list,
                                    train_dataset=train_dataset,
                                    tokenizer=tokenizer,
                                    fold_results=fold_results,
                                    cluster_id=cluster_idx)

cv_df = results_dict_todf(fold_results)

########## SAVE THE FILE

with open(f'reports/results/cv_result_{ANALYSIS_POSTFIX}.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Filter:   0%|          | 0/5291 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5291 [00:00<?, ? examples/s]

Map:   0%|          | 0/5291 [00:00<?, ? examples/s]

Fold 0


Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/2651 [00:00<?, ? examples/s]

Map:   0%|          | 0/2651 [00:00<?, ? examples/s]

TRAINING CLUSTER SET 3 FOR EPOCHS1


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


In [12]:
fold_results.keys()

dict_keys([0, 1, 5, 8, 10, 16, 'cluster_1', 'cluster_4', 'cluster_3'])

In [13]:
print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())

Mean
model_set
0            0.113737
1            0.247452
5            0.273484
8            0.273170
10           0.270033
16           0.268677
cluster_1    0.154332
cluster_3    0.151154
cluster_4    0.149859
Name: rouge, dtype: float64
STD
model_set
0            0.124526
1            0.157347
5            0.158682
8            0.158973
10           0.158828
16           0.158742
cluster_1    0.148038
cluster_3    0.146543
cluster_4    0.145705
Name: rouge, dtype: float64


In [6]:
########## LOAD CV RESULTS

import pickle
with open(f'reports/results/cv_result_{ANALYSIS_POSTFIX}.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

### Step 2. Learn performance

In [ ]:
cv_df, model_results = cv_step_2(experiment_config=experiment_config, cv_df=cv_df)

with open(f'reports/results/s2_model_results_{ANALYSIS_POSTFIX}.pickle', 'wb') as handle:
    pickle.dump(model_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'reports/results/cd_df_with_s2_{ANALYSIS_POSTFIX}.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())

0
lr
svm


In [4]:
### TO SAVE THE VECTORIZER AND STEP 2 MODELS

with open(f'reports/results/cd_df_with_s2_{ANALYSIS_POSTFIX}.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

In [5]:
full_step_2(cv_df=cv_df, 
            experiment_config=experiment_config)

lr
svm
lgbm
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 4.989763 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21187
[LightGBM] [Info] Number of data points in the train set: 71478, number of used features: 1422
[LightGBM] [Info] Start training from score 0.211322
catboost
Learning rate set to 0.080378
0:	learn: 0.1619674	total: 68.3ms	remaining: 1m 8s
1:	learn: 0.1603686	total: 82.6ms	remaining: 41.2s
2:	learn: 0.1590035	total: 94.8ms	remaining: 31.5s
3:	learn: 0.1578352	total: 106ms	remaining: 26.4s
4:	learn: 0.1568431	total: 116ms	remaining: 23.1s
5:	learn: 0.1559708	total: 127ms	remaining: 21s
6:	learn: 0.1552563	total: 137ms	remaining: 19.5s
7:	learn: 0.1545913	total: 148ms	remaining: 18.4s
8:	learn: 0.1540291	total: 159ms	remaining: 17.5s
9:	learn: 0.1535557	total: 170ms	remaining: 16.8s
10:	learn: 0.1531613	total: 180ms	remai